In [1]:
import pandas as pd

In [3]:
cars = pd.read_csv("data/car_0.csv")

likes_list = [pd.read_csv("data/like_0.csv"), pd.read_csv("data/like_2.csv"), pd.read_csv("data/like_3.csv"), pd.read_csv("data/like_4.csv"),]
users_list = [pd.read_csv("data/user_0.csv"), pd.read_csv("data/user_2.csv"), pd.read_csv("data/user_3.csv"), pd.read_csv("data/user_4.csv"),]





like = pd.concat(likes_list, axis=0, ignore_index=True)
user = pd.concat(users_list, axis=0, ignore_index=True)

## Idea

Combining session based (sequential recommendations) with car data (representation)
We represent the cars as a vector of size 32
the user's ideal car (propagating vector in the RNN) is set to the users demands initially

In [4]:
cars.head()

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,doors,seats,fiscalPower,DINPower,critAir,nb_img,link,id
0,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,12890,2017,60933,Essence,Manuelle,Blanc,5,5.0,4,82,1.0,7,NaN,2
1,RENAULT,Talisman,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,24900,2019,63510,Essence,Automatique,Gris,5,5.0,8,159,1.0,20,NaN,3
2,RENAULT,Kadjar,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,20890,2019,58858,Diesel,Manuelle,Gris,5,5.0,6,116,2.0,20,NaN,4
3,RENAULT,Koleos,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,36490,2022,64,Essence,Automatique,Noir,5,5.0,8,158,1.0,20,NaN,5
4,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,14890,2021,42555,Essence,Manuelle,Gris,5,5.0,4,83,1.0,20,NaN,6


In [5]:
vectorized_cars = {}

In [6]:
parameter_weights = {"brand":2, "model":2,"price":4, "year": 2, "km": 3, "fuel": 5, "gearbox":4, "color": 3, "doors": 1, "seats": 1, "fiscalPower": 2, "DINPower": 2, "critAir": 1}

In [7]:
print("This number has to be exactly equal to 32 :")
sum(parameter_weights.values())

This number has to be exactly equal to 32 :


32

In [8]:
def map_to_num(dataf, colname):
    mapping = {item:i for i, item in enumerate(dataf[colname].unique())}
    dataf[colname+"_n"] = dataf[colname].apply(lambda x: mapping[x])
    return mapping

In [9]:
textual_data = ["brand", "model", "fuel", "gearbox", "color"]

In [10]:
mapping_dict = {}

In [11]:
for param in textual_data:
  mapping_dict[param] = map_to_num(cars, param)

Note for the future, since color takes 3 places, we can actually just use the RGB, maybe get it from the image(majority color)

In [11]:
cars.head()

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,...,fiscalPower,DINPower,critAir,nb_img,id,brand_n,model_n,fuel_n,gearbox_n,color_n
0,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,12890,2017,60933,Essence,Manuelle,Blanc,...,4,82,1.0,7,2,0,0,0,0,0
1,RENAULT,Talisman,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,24900,2019,63510,Essence,Automatique,Gris,...,8,159,1.0,20,3,1,1,0,1,1
2,RENAULT,Kadjar,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,20890,2019,58858,Diesel,Manuelle,Gris,...,6,116,2.0,20,4,1,2,1,0,1
3,RENAULT,Koleos,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,36490,2022,64,Essence,Automatique,Noir,...,8,158,1.0,20,5,1,3,0,1,2
4,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,14890,2021,42555,Essence,Manuelle,Gris,...,4,83,1.0,20,6,0,0,0,0,1


In [12]:
cars["price"] = cars["price"] / 1000

In [13]:
cars["km"] = cars["km"] / 10000
cars["year"] = cars["year"] - 1900

In [14]:
cars.head()

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,...,DINPower,critAir,nb_img,link,id,brand_n,model_n,fuel_n,gearbox_n,color_n
0,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,12.89,117,6.0933,Essence,Manuelle,Blanc,...,82,1.0,7,NaN,2,0,0,0,0,0
1,RENAULT,Talisman,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,24.90,119,6.3510,Essence,Automatique,Gris,...,159,1.0,20,NaN,3,1,1,0,1,1
2,RENAULT,Kadjar,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,20.89,119,5.8858,Diesel,Manuelle,Gris,...,116,2.0,20,NaN,4,1,2,1,0,1
3,RENAULT,Koleos,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,36.49,122,0.0064,Essence,Automatique,Noir,...,158,1.0,20,NaN,5,1,3,0,1,2
4,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,14.89,121,4.2555,Essence,Manuelle,Gris,...,83,1.0,20,NaN,6,0,0,0,0,1


In [15]:
import numpy as np

In [16]:
car_vectors = {}

In [17]:
for index, row in cars.iterrows():
  car_vector = np.ndarray((32,))
  i = 0
  for param, weight in parameter_weights.items():
    if param in textual_data:
      car_vector[i:i+weight] = row[param+"_n"]
    else :
      car_vector[i:i+weight] = row[param]
    i += weight
  car_vectors[row["id"]] = car_vector

# Data Preparation

In [18]:
like.head()

,user_id,car_id,value,swiped
0,57,2,-1,t
1,29,3,-1,f
2,29,4,-1,f
3,29,5,-1,f
4,29,6,-1,f


In [19]:
user.head()

,maxPrice,essence,diesel,hybrid,elec,manual,auto,id
0,25000,t,f,t,t,t,t,57
1,-1,t,f,f,f,t,f,27
2,25000,t,f,t,t,t,t,28
3,13000,t,t,f,f,t,f,29
4,64000,t,t,f,f,f,t,30


In [20]:
user_context_vectors = {}

In [21]:
for index, row in user.iterrows():
  user_vector = np.array([0]*32)
  user_vector[3:3+4] = row["maxPrice"] / 1000
  user_vector[12] = int(row["essence"] == "t")
  user_vector[13] = int(row["diesel"]  == "t")
  user_vector[14] = int(row["hybrid"]  == "t")
  user_vector[15] = int(row["elec"] == "t")

  user_vector[17:19] = int(row["manual"]  == "t")
  user_vector[19:21] = int(row["auto"]  == "t")
  user_context_vectors[row["id"]] = user_vector

In [22]:
inputs = []
outputs = []
current_context_vectors= user_context_vectors.copy()

In [23]:
like.head()

,user_id,car_id,value,swiped
0,57,2,-1,t
1,29,3,-1,f
2,29,4,-1,f
3,29,5,-1,f
4,29,6,-1,f


In [25]:
for index,inter in like.iterrows():
  ideal_cars = like.iloc[index:][like["user_id"] == inter["user_id"]][like["value"] == 0] 
  if ideal_cars.shape[0] > 0:
    if not (inter["user_id"] in current_context_vectors.keys()) :
      user_context = np.array([0]*32)
    else:
      user_context = current_context_vectors[inter["user_id"]]
    inputs.append(np.concatenate((car_vectors[inter["car_id"]], np.array([inter["value"]]*16) ,user_context )))
    outputs.append(car_vectors[ideal_cars.iloc[0]["car_id"]])
  if inter["value"] == 0:
    current_context_vectors[inter["user_id"]] = car_vectors[inter["car_id"]]
    


C:\Users\Abderrahmane\AppData\Local\Temp\ipykernel_10288\2747314896.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_cars = like.iloc[index:][like["user_id"] == inter["user_id"]][like["value"] == 0]


In [72]:
len(inputs) == len(outputs)

True

In [73]:
len(inputs)

13805

In [74]:
len(outputs[0])

32

In [84]:
inputs = np.nan_to_num(inputs)

In [85]:
outputs = np.nan_to_num(outputs)

# Building the neural network

In [86]:
import tensorflow as tf

In [87]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(64)),
        tf.keras.layers.Dense(64, activation="relu", name="Dense1"),
        tf.keras.layers.Dense(128, activation="relu", name="Dense2"),
        tf.keras.layers.Dense(128, activation="relu", name="Dense3"),
        tf.keras.layers.Dense(64, activation="relu", name="Dense4"),
        tf.keras.layers.Dense(32, activation="relu", name="DenseOutput"),
    ]
)

In [88]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense1 (Dense)              (None, 64)                4160      
                                                                 
 Dense2 (Dense)              (None, 128)               8320      
                                                                 
 Dense3 (Dense)              (None, 128)               16512     
                                                                 
 Dense4 (Dense)              (None, 64)                8256      
                                                                 
 DenseOutput (Dense)         (None, 32)                2080      
                                                                 
Total params: 39,328
Trainable params: 39,328
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.CosineSimilarity(),
    # List of metrics to monitor
    metrics=[tf.keras.metrics.Precision()],
)

In [90]:
train_dataset = tf.data.Dataset.from_tensor_slices((inputs[:int(len(inputs)*3/4)],outputs[:int(len(outputs)*3/4)]))
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset =  tf.data.Dataset.from_tensor_slices((inputs[int(len(inputs)*3/4):],outputs[int(len(outputs)*3/4):]))

In [91]:
model.fit(train_dataset, batch_size=64, epochs=11)

Epoch 1/11
162/162 [==============================] - 2s 7ms/step - loss: -0.9598 - precision_5: 0.9613
Epoch 2/11
162/162 [==============================] - 1s 9ms/step - loss: -0.9701 - precision_5: 0.9721
Epoch 3/11
162/162 [==============================] - 1s 7ms/step - loss: -0.9724 - precision_5: 0.9712
Epoch 4/11
162/162 [==============================] - 1s 7ms/step - loss: -0.9727 - precision_5: 0.9759
Epoch 5/11
162/162 [==============================] - 3s 16ms/step - loss: -0.9745 - precision_5: 0.9745
Epoch 6/11
162/162 [==============================] - 1s 8ms/step - loss: -0.9750 - precision_5: 0.9737
Epoch 7/11
162/162 [==============================] - 1s 8ms/step - loss: -0.9752 - precision_5: 0.9717
Epoch 8/11
162/162 [==============================] - 1s 8ms/step - loss: -0.9758 - precision_5: 0.9711
Epoch 9/11
162/162 [==============================] - 1s 8ms/step - loss: -0.9760 - precision_5: 0.9705
Epoch 10/11
162/162 [==============================] - 1s 7ms/s

In [92]:
model.save("../base_model")

INFO:tensorflow:Assets written to: ../base_model\assets


In [93]:
model.evaluate(test_dataset.batch(BATCH_SIZE))

54/54 [==============================] - 0s 5ms/step - loss: -0.9727 - precision_5: 0.9798


[-0.9727262854576111, 0.9798335433006287]